In [1]:
import requests
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from scipy.cluster.vq import whiten, kmeans2

In [3]:
#API token: 2502c6aa7eb148a989cfe4b3b98ca5ae

In [4]:
def call_api(uri, result_key):
    json = {}
    try:
        json = requests.get(uri, headers = { 'X-Auth-Token': '2502c6aa7eb148a989cfe4b3b98ca5ae' }).json()
        return json[result_key]
    except:
        print(json)
        return None

In [5]:
teams = call_api('http://api.football-data.org/v1/soccerseasons/439/teams', 'teams')
teams

[{'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/496/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/496/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/496'}},
  'code': None,
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/de/6/63/Rio_Ave_FC.svg',
  'name': 'FC Rio Ave',
  'shortName': 'Rio Ave',
  'squadMarketValue': '21,125,000 €'},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/503/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/503/players'},
   'self': {'href': 'http://api.football-data.org/v1/teams/503'}},
  'code': 'FCP',
  'crestUrl': 'http://upload.wikimedia.org/wikipedia/de/e/ed/FC_Porto_1922-2005.svg',
  'name': 'FC Porto',
  'shortName': 'Porto',
  'squadMarketValue': '204,900,000 €'},
 {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/583/fixtures'},
   'players': {'href': 'http://api.football-data.org/v1/teams/583/players

In [ ]:
for team in teams:
    market_value = team['squadMarketValue']
    fixtures = call_api(team['_links']['fixtures']['href'], 'fixtures')
    losses = 0
    for f in fixtures:
        if f['result'] is None or f['result']['goalsHomeTeam'] is None or f['result']['goalsAwayTeam'] is None:
            continue
        if team['name'] == f['homeTeamName']:
            losses += 1 if f['result']['goalsHomeTeam'] < f['result']['goalsAwayTeam'] else 0
        else:
            losses += 1 if f['result']['goalsAwayTeam'] < f['result']['goalsHomeTeam'] else 0
    print('Team "{}", € {} market value, {} losses'.format(team['name'], market_value, losses))
    team['losses'] = losses

In [ ]:
team_for_pd = []
for team in teams:
    if team['squadMarketValue']:
        team_for_pd.append({'name': team['name'], 'market_value': int(re.sub('\D', '', team['squadMarketValue'])), 'losses': team['losses']})
team_for_pd

In [ ]:
teams_pd = pd.DataFrame(team_for_pd)
teams_pd

In [ ]:
teams_pd.plot.scatter(x='market_value', y='wins', s=100)

In [ ]:
del teams_pd['name']
teams_pd

In [ ]:
whitened = whiten(teams_pd.as_matrix())
whitened

In [ ]:
centroids, cluster_map = kmeans2(whitened, 3)
centroids, cluster_map

In [ ]:
colors_map = {0: 'r', 1: 'g', 2: 'b', 3: 'y', 4: 'm'}
colors = [colors_map[c] for c in cluster_map]
colors

In [ ]:
teams_pd.plot.scatter(x='market_value', y='wins', c=colors, s=100)

In [ ]:
centroids, cluster_map = kmeans2(whitened, 5)
centroids, cluster_map

In [ ]:
colors_map = {0: 'r', 1: 'g', 2: 'b', 3: 'y', 4: 'm'}
colors = [colors_map[c] for c in cluster_map]
colors

In [ ]:
teams_pd.plot.scatter(x='market_value', y='wins', c=colors, s=100)